In [25]:
import numpy as np 
import torch
import torch.utils.data as Data
import torch.optim as optim

from MyNet import Net, MetaDataSet, MyLoss1, MyLoss, Net_3hiddens

# hyper parameters
BATCH_SIZE = 1000
EPOCH_SIZE = 50

LR = 1e-2
MOMENTUM = 0.9
WEIGHT_DECAY =  0.0005

DATASET_SHUFFLE = False
num_WORKERS = 3

TASK = 'regression'

LAMBDA = 1
XI = 0.0005


save_path = './'
# metadata_dir = 'E:/metadata数据集/new_bigmetadata/australian/query_time/'
metadata_dir = 'D:/generate_metadata/bigmetadata/australian/model1/query_time50/'


# net = Net(n_feature=396, first_n_hidden=100, second_n_hidden=20, n_output=2)
# net_bn = Net(n_feature=418, first_n_hidden=200, second_n_hidden=100, n_output=1, batch_normalization=True)
# net_no_bn = Net(n_feature=418, first_n_hidden=200, second_n_hidden=100, n_output=1, batch_normalization=False)
net3_bn = Net_3hiddens(n_feature=396, first_n_hidden=200, second_n_hidden=100, third_n_hidden=50, n_output=1, batch_normalization=True)
net3_no_bn = Net_3hiddens(n_feature=396, first_n_hidden=200, second_n_hidden=100, third_n_hidden=50, n_output=1, batch_normalization=False)

nets = [net3_bn, net3_no_bn]
# optimizers = [optim.Adam(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY) for net in nets]
optimizers = [optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY) for net in nets]

criterion = MyLoss(XI)
mse_loss = torch.nn.MSELoss()
print(nets)
print(optimizers)
print(criterion)

[Net_3hiddens(
  (input_bn): BatchNorm1d(396, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (first_hidden): Linear(in_features=396, out_features=200, bias=True)
  (first_bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (second_hidden): Linear(in_features=200, out_features=100, bias=True)
  (second_bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (third_hidden): Linear(in_features=100, out_features=50, bias=True)
  (third_bn): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=50, out_features=1, bias=True)
), Net_3hiddens(
  (input_bn): BatchNorm1d(396, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (first_hidden): Linear(in_features=396, out_features=200, bias=True)
  (first_bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (second_hidden): Linear(in_features=200, out_features

In [26]:
# print('Preparing the dataset....')
# loader = MetaDataSet(metadata_dir, BATCH_SIZE, num_WORKERS, DATASET_SHUFFLE, TASK)
# print("Dataset is ready!")

metadata = np.load('D:/generate_metadata/bigmetadata/australian/model1/query_time50/2australian30_big_metadata50.npy')
# metadata2 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/4australian30_big_metadata30.npy')
# metadata3 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/10australian30_big_metadata30.npy')
# metadata4 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/20australian30_big_metadata30.npy')
# metadata5 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/30australian30_big_metadata30.npy')
# metadata6 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/48australian30_big_metadata30.npy')

# metadata = np.vstack((metadata1, metadata2, metadata3, metadata4, metadata5, metadata6))

X = metadata[:, 0:396]
y = metadata[:, 396]

X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)
X_tensor = X_tensor.float()
y_tensor = y_tensor.float()

troch_dataset = Data.TensorDataset(X_tensor, y_tensor)
loader = Data.DataLoader(dataset=troch_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=DATASET_SHUFFLE,
                        num_workers=num_WORKERS)

In [24]:
# tarining the NN
for epoch in range(200):
    running_loss0 = 0.0
    running_loss1 = 0.0
    for step, (batch_x, batch_y) in enumerate(loader):
#         print('step',step)
        min_b = int(BATCH_SIZE/2)
        pari_y = batch_y[0:min_b] - batch_y[min_b:]
#         pari_y = 
#         if step % 20 == 19:    
#             print('min pari_y',min(pari_y))
#             print('max pari_y',max(pari_y))
#             print('abs min pari_y',min(abs(pari_y)))
#             print('abs max pari_y',max(abs(pari_y)))
        for net, optimizer in zip(nets, optimizers):
            optimizer.zero_grad()
            output1 = net(batch_x[0:min_b,:])
            output2 = net(batch_x[min_b:,:])
            loss = criterion(output1, output2, pari_y) + mse_loss(torch.cat((output1, output2))[:,0], batch_y)
#             loss = mse_loss(torch.cat((output1, output2))[:,0], batch_y)
#             print(loss)

#             print('the dtype of loss ', loss.dtype)
#             print('the shape of loss', loss.size())
#             print('loss ', loss)
            loss.backward()
            optimizer.step()
            
            if net == nets[0]:
                i = 0
                # print statistics
                running_loss0 += loss.item()
                # print every 2000 mini-batches
                if step % 20 == 19:    
                    print('net',i,' [%d, %5d] loss: %.3f' %
                        (epoch + 1, step + 1, running_loss0 / 2000))
                    running_loss0 = 0.0
            else:
                i = 1
                # print statistics
                running_loss1 += loss.item()
                # print every 2000 mini-batches
                if step % 20 == 19:    
                    print('net',i,' [%d, %5d] loss: %.3f' %
                        (epoch + 1, step + 1, running_loss1 / 2000))
                    running_loss1 = 0.0          

from sklearn.metrics import r2_score, accuracy_score
# ranking accuracy.
net2 = nets[1]
# print(net2)
n_sample = metadata.shape[0]
X = metadata[:, 0:396]
X_tensor = torch.from_numpy(X)
X_tensor = X_tensor.float()
y = metadata[:, 396]
print('y-------',y)
# pair_X = [np.hstack((X[i,:],X[i+1,:])) for i in range(0, n_sample, 2)]
pair_y = [(y[i]-y[i+1]) for i in range(0, n_sample, 2)]
pair_y = np.array(pair_y)
pair_y[np.where(pair_y>0)[0]] = 1
pair_y[np.where(pair_y<=0)[0]] = 0
out = net2(X_tensor)

pred = out.detach().numpy()
print('pred**********',pred)
pred = pred[:,0]
print('r2 score : ',r2_score(y, pred))
pred = [(pred[i]-pred[i+1]) for i in range(0, n_sample, 2)]
pred = np.array(pred)
pred[np.where(pred>0)[0]] = 1
pred[np.where(pred<=0)[0]] = 0
print(np.shape(pair_y))
# pair_X_tensor = torch.from_numpy(pair_X)
# pair_y_tensor = torch.from_numpy(pair_y)
print(accuracy_score(pair_y, pred))

net 0  [1,    20] loss: nan
net 1  [1,    20] loss: nan
net 0  [2,    20] loss: nan
net 1  [2,    20] loss: nan
net 0  [3,    20] loss: nan
net 1  [3,    20] loss: nan


KeyboardInterrupt: 

In [ ]:
# save the parameters in NN
torch.save({
            'model_state_dict': nets[0].state_dict(),
            'optimizer_state_dict': optimizers[0].state_dict(),
            }, './pairloss_bn_net_parameters.pth')
torch.save({
            'model_state_dict': nets[1].state_dict(),
            'optimizer_state_dict': optimizers[1].state_dict(),
            }, './pairloss_nobn_net_parameters.pth')  

In [6]:
# 测试数据集
from sklearn.metrics import r2_score, accuracy_score

metadata1 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/2australian30_big_metadata30.npy')
metadata2 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/4australian30_big_metadata30.npy')
metadata3 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/10australian30_big_metadata30.npy')
metadata4 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/20australian30_big_metadata30.npy')
metadata5 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/30australian30_big_metadata30.npy')
metadata6 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/48australian30_big_metadata30.npy')

metadata = np.vstack((metadata1, metadata2, metadata3, metadata4, metadata5, metadata6 ))

In [7]:
# 普通回归测试
net2 = nets[1]
# X = metadata[:, 0:396]
# y = metadata[:, 396]
X = metadata[:, 0:418]
y = metadata[:, 418]
print(y.shape)
X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)
X_tensor = X_tensor.float()
y_tensor = y_tensor.float()
out = net2(X_tensor)
print(out.size())
pred = out.detach().numpy()
print(pred.shape)
print(pred[:,0].shape)
# print(predict)
# print(accuracy_score(y, predict))
print(r2_score(y, pred[:,0]))
y[np.where(y>0)[0]] = 1
y[np.where(y<=0)[0]] = 0
cperd = pred[:,0]
cperd[np.where(cperd>0)[0]] = 1
cperd[np.where(cperd<=0)[0]] = 0
print(accuracy_score(y, cperd))

(162000,)
torch.Size([162000, 1])
(162000, 1)
(162000,)
-0.03237689474503469
0.5459259259259259


In [19]:
# ranking accuracy.
net2 = nets[1]
# print(net2)
n_sample = metadata.shape[0]
n_sample = 10
X = metadata[0:12, 0:418]
X_tensor = torch.from_numpy(X)
X_tensor = X_tensor.float()
print(X_tensor)
y = metadata[:, 418]
y = torch.from_numpy(y)
# pair_X = [np.hstack((X[i,:],X[i+1,:])) for i in range(0, n_sample, 2)]
b1_x = [X_tensor[i,:] for i in range(0,n_sample, 2)]
b2_x = [X_tensor[i+1,:] for i in range(0,n_sample, 2)]
print(b1_x)
b1_x = torch.tensor(b1_x)
b2_x = torch.tensor(b2_x)

print(b1_x.size())
print(b2_x.size())

pair_y = [(y[i]-y[i+1]) for i in range(0, n_sample, 2)]
print(pari_y.size())
pair_y = np.array(pair_y)
pair_y[np.where(pair_y>0)[0]] = 1
pair_y[np.where(pair_y<=0)[0]] = 0
out = net2(X_tensor)

pred = out.detach().numpy()
pred = pred[:,0]
pred = [(pred[i]-pred[i+1]) for i in range(0, n_sample, 2)]
pred = np.array(pred)
pred[np.where(pred>0)[0]] = 1
pred[np.where(pred<=0)[0]] = 0
print(np.shape(pair_y))
# pair_X_tensor = torch.from_numpy(pair_X)
# pair_y_tensor = torch.from_numpy(pair_y)
print(accuracy_score(pair_y, pred))

tensor([[ 6.9000e+02,  6.5367e+00,  4.2000e+01,  ..., -4.4707e-01,
         -5.0195e-01, -6.1290e-01],
        [ 6.9000e+02,  6.5367e+00,  4.2000e+01,  ..., -1.0543e-01,
         -1.5824e-01, -2.1628e-01],
        [ 6.9000e+02,  6.5367e+00,  4.2000e+01,  ...,  1.0560e-01,
          4.7534e-02, -9.0591e-03],
        ...,
        [ 6.9000e+02,  6.5367e+00,  4.2000e+01,  ..., -1.2116e-01,
         -1.9615e-01, -2.6588e-01],
        [ 6.9000e+02,  6.5367e+00,  4.2000e+01,  ...,  1.6786e-01,
          1.1049e-01,  4.1618e-02],
        [ 6.9000e+02,  6.5367e+00,  4.2000e+01,  ..., -6.1331e-01,
         -6.7973e-01, -7.7335e-01]])
[tensor([ 6.9000e+02,  6.5367e+00,  4.2000e+01,  3.7377e+00,  6.0870e-02,
        -2.7990e+00,  1.6429e+01,  2.7990e+00, -2.7784e+00,  1.8493e+01,
         4.8629e+00,  5.4844e+00, -1.9914e+00,  3.4000e+02,  5.3920e+01,
         9.6326e+01,  9.5317e-01,  1.1570e+00, -4.7184e-01,  9.2812e-02,
        -2.5425e-01,  6.2991e-01,  4.2000e+01,  5.0000e-01,  5.0000e-01,
  

         1.2079e-01,  4.7284e-02, -3.0774e-02])]


ValueError: only one element tensors can be converted to Python scalars

In [1]:
import os
import numpy as np 
import torch
import torch.utils.data as Data
import torch.optim as optim

from sklearn.metrics import r2_score, accuracy_score

from MyNet import Net, MetaDataSet, MyLoss, Net_3hiddens

# hyper parameters
BATCH_SIZE = 128
EPOCH_SIZE = 2

LR = 1e-4
MOMENTUM = 0.9
WEIGHT_DECAY =  0.0005

DATASET_SHUFFLE = False
num_WORKERS = 3

TASK = 'regression'

LAMBDA = 1
XI = 0.0005


save_path = './'
metadata_dir = 'E:/metadata数据集/new_bigmetadata/australian/query_time/'

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# net = Net(n_feature=396, first_n_hidden=100, second_n_hidden=20, n_output=2)
# net_bn = Net(n_feature=418, first_n_hidden=200, second_n_hidden=100, n_output=1, batch_normalization=True)
# net_no_bn = Net(n_feature=418, first_n_hidden=200, second_n_hidden=100, n_output=1, batch_normalization=False)
net = Net_3hiddens(n_feature=418, first_n_hidden=200, second_n_hidden=100, third_n_hidden=50, n_output=1, batch_normalization=True)

# optimizer = optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
optimizer = optim.Adam(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = MyLoss(XI, LAMBDA)
print(net)
print(optimizer)
print(criterion)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('The current device is : ', device)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)

print('Preparing the dataset....')
# trainloader = MetaDataSet(metadata_dir, BATCH_SIZE, num_WORKERS, DATASET_SHUFFLE, TASK)
# testloader = MetaDataSet(metadata_dir, BATCH_SIZE, num_WORKERS, DATASET_SHUFFLE, TASK)

metadata1 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/2australian30_big_metadata30.npy')
metadata2 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/4australian30_big_metadata30.npy')
metadata3 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/10australian30_big_metadata30.npy')
metadata4 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/20australian30_big_metadata30.npy')
metadata5 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/30australian30_big_metadata30.npy')
metadata6 = np.load('D:/generate_metadata/new_bigmetadata/australian/query_time/48australian30_big_metadata30.npy')

metadata = np.vstack((metadata1, metadata2, metadata3, metadata4, metadata5, metadata6))

X = metadata[:, 0:418]
y = metadata[:, 418]

X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)
X_tensor = X_tensor.float()
y_tensor = y_tensor.float()

troch_dataset = Data.TensorDataset(X_tensor, y_tensor)
trainloader = Data.DataLoader(dataset=troch_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=DATASET_SHUFFLE,
                        num_workers=num_WORKERS)

test_X = metadata1[:, 0:418]
test_y = metadata1[:, 418]

test_X_tensor = torch.from_numpy(X)
test_y_tensor = torch.from_numpy(y)
test_X_tensor = X_tensor.float()
test_y_tensor = y_tensor.float()

test_troch_dataset = Data.TensorDataset(X_tensor, y_tensor)
testloader = Data.DataLoader(dataset=test_troch_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=DATASET_SHUFFLE,
                        num_workers=num_WORKERS)

print("Dataset is ready!")

        


Net_3hiddens(
  (input_bn): BatchNorm1d(418, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (first_hidden): Linear(in_features=418, out_features=200, bias=True)
  (first_bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (second_hidden): Linear(in_features=200, out_features=100, bias=True)
  (second_bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (third_hidden): Linear(in_features=100, out_features=50, bias=True)
  (third_bn): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=50, out_features=1, bias=True)
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0.0005
)
MyLoss()
The current device is :  cuda
Preparing the dataset....


FileNotFoundError: [Errno 2] No such file or directory: 'D:/generate_metadata/new_bigmetadata/australian/query_time/2australian30_big_metadata30.npy'

In [14]:
def train(epoch):
    print('\nEpoch: %d' % (epoch+1))
    net.train()
    # 
    if epoch < EPOCH_SIZE * 0.3:
        lam = 0
        xi = 0.0005
    elif epoch < EPOCH_SIZE * 0.6:
        lam = 0.5
        xi = 0.0005
    else:
        lam = 1
        xi = 0.0001       
    running_loss = 0.0
    for step, (batch_x, batch_y) in enumerate(trainloader):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
#         min_b = int(BATCH_SIZE/2)
        min_b = int(batch_y.size()[0]/2)

        pari_y = batch_y[0:min_b] - batch_y[min_b:]
        optimizer.zero_grad()
        output1 = net(batch_x[0:min_b,:])
        output2 = net(batch_x[min_b:,:])
        loss = criterion(output1, output2, pari_y, batch_y, xi, lam)
        loss.backward()
        optimizer.step()   
  
        # print statistics
        running_loss += loss.item()
        # print every 1000 mini-batches
        if step % 1000 == 999:    
            print('[%d, %5d] ,progress %.4f, mean loss: %.10f' %
                (epoch + 1, step + 1, step/len(trainloader), running_loss / 1000))
            running_loss = 0.0

def test(epoch):
    global best_acc
    net.eval()
    r2scorelist = []
    acclist = []
    with torch.no_grad():
        for step, (batch_x, batch_y) in enumerate(testloader):
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    #         min_b = int(BATCH_SIZE/2)
            min_b = int(batch_y.size()[0]/2)
            pari_y = batch_y[0:min_b] - batch_y[min_b:]
            pari_y[np.where(pari_y>0)[0]] = 1
            pari_y[np.where(pari_y<=0)[0]] = 1
            output = net(batch_x)
            pred = output.detach().numpy()
            y = batch_y.detach().numpy()
            r2score = r2_score(pred, y)
            pair_pred = pred[0:min_b] - pred[min_b:]
            pair_pred[np.where(pair_pred>0)[0]] = 1
            pair_pred[np.where(pair_pred<=0)[0]] = 1
            acc = accuracy_score(pair_pred, pari_y)
            r2scorelist.append(r2score)
            acclist.append(acc)

    mean_acc = np.mean(acclist)
    mean_r2 = np.mean(r2scorelist)
    # Save checkpoint.
    if mean_acc > best_acc:
        print('Saving..')
        print('Current mean_rank_acc: %.10f'%mean_acc)
        print('Current mean r2 score: %.10f'%mean_r2)
        state = {
            'net': net.state_dict(),
            'mean_rank_acc': mean_acc,
            'mean_r2_score':mean_r2,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = mean_acc
        
for epoch in range(4):
    train(epoch)
    test(epoch)


Epoch: 1
[1,  1000] ,progress 0.7891, mean loss: 0.0020050095
Saving..
Current mean_rank_acc: 1.0000000000
Current mean r2 score: -3.6074960414

Epoch: 2
[2,  1000] ,progress 0.7891, mean loss: nan


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').